# Problem
It is not able to learn from the features (loss remains the same)

# To-Do
- Test the random init of the weights
- F1 Score
- plot learning curve (loss per iteration)
- Dropout
- Confusion Matrix

In [ ]:
from subprocess import check_output # system functions
import numpy as np # linear algebra
import pandas as pd # data processing
import matplotlib.pyplot as plt # visualize data
from imblearn.under_sampling import RandomUnderSampler # random undersampling
from imblearn.over_sampling import RandomOverSampler # random oversampling
from sklearn.model_selection import train_test_split # train / test split
from keras.models import Sequential
from keras.layers import Activation, Dense

In [ ]:
# list the input directory
print(check_output(['ls', '../input']).decode('utf8'))

# load the data into a dataframe
df = pd.read_csv('../input/creditcard.csv')

In [ ]:
# show the first examples
df[0:5]

In [ ]:
# rename a column
df = df.rename(index=str, columns={'Class': 'Fraud'})

# show the unbalace of the data 
count = pd.value_counts(df['Fraud'], sort = True).sort_index()
count.plot(kind = 'bar')
plt.xlabel('Class')
plt.ylabel('Frequency')

In [ ]:
# split the data to features / prediction class
X_data_ = df.drop(['Fraud'], axis=1)
Y_data_ = df['Fraud']

# normalization without the labels (0 and 1)
X_data_ = (X_data_ - X_data_.mean()) / (X_data_.max() - X_data_.min())

# show the normalized data
print(X_data_[0:3])

# random oversampling
# ros = RandomOverSampler(random_state=0)
# X_data, Y_data = ros.fit_sample(X_data_, Y_data_)

# random undersampling
rus = RandomUnderSampler(return_indices=True)
X_data, Y_data, idx_resampled = rus.fit_sample(X_data_, Y_data_)

# show the balance of the data
count = pd.value_counts(Y_data, sort = True).sort_index()
count.plot(kind = 'bar')
plt.xlabel('Labels')
plt.ylabel('Frequency')

In [ ]:
# train / test split
X_train, X_test, y_train, y_test = train_test_split(X_data, Y_data, test_size=0.33, random_state=42)

# show the dimensions of the train / test data
print("X_train.shape: ", X_train.shape)
print("X_test.shape: ", X_test.shape)
print("y_train.shape: ", y_train.shape)
print("y_test.shape: ", y_test.shape)

In [ ]:
# hyper parameters
learning_rate = 0.01
batch_size = 32
epochs = 200

# fully-connected neural network
model = Sequential()
model.add(Dense(32, input_shape=X_train.shape[1:], kernel_initializer='random_uniform', bias_initializer='zeros'))
model.add(Activation('relu'))
model.add(Dense(64, kernel_initializer='random_uniform'))
model.add(Activation('relu'))
model.add(Dense(1, kernel_initializer='random_uniform'))
model.add(Activation('softmax'))

# training error
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['mae','acc'])

In [ ]:
# model training
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=0)
score = model.evaluate(X_test, y_test, verbose=0)

In [ ]:
print('Test loss:', score[0])
print('Test accuracy:', score[1])